# Processing GRB files from EUMETSAT
Data from geostationary satellite was downloaded to have maximum control over the analysis. Open-Meteo weather API may be an easier alternative for future studies.

In [ ]:
import os
import pygrib
import numpy as np
import pandas as pd
from tqdm import tqdm

# ---- CONFIG ---- #

# The folder containing all your .grb files
GRIB_FOLDER = "/absolute/path/to/your/grib/files"

# The location where the final analysis CSV will be saved
OUTPUT_CSV_PATH = "/absolute/path/to/your/output/folder/cloud_analysis.csv"

# The geographic coordinates you want to analyze
TARGET_LAT = 36.824167
TARGET_LON = 30.335556

# The size of the box to check for clouds around the target pixel.
# A size of 1 means a 1x1 box (just the center pixel).
# A size of 3 means a 3x3 box centered on the target pixel.
ANALYSIS_BOX_SIZE = 3


# ---- FUNCTIONS ---- #

def find_closest_pixel(lats, lons, target_lat, target_lon):
    """
    Finds the closest grid point in the lat/lon arrays to a target coordinate.

    Args:
        lats (np.array): 2D array of latitudes from the GRIB file.
        lons (np.array): 2D array of longitudes from the GRIB file.
        target_lat (float): The target latitude.
        target_lon (float): The target longitude.

    Returns:
        tuple: The (row, column) index of the closest pixel.
    """
    # The EUMETSAT data seems to use positive values for West longitudes
    target_lon_fixed = -1 * target_lon

    # Simple squared distance is fast and sufficient for finding the minimum
    dist_sq = (lats - target_lat)**2 + (lons - target_lon_fixed)**2
    min_index_flat = np.argmin(dist_sq)
    
    # Convert the 1D index back to a 2D (row, col) index
    return np.unravel_index(min_index_flat, lats.shape)


def analyze_grib_files(folder_path, center_row, center_col, box_size):
    """
    Processes all GRIB files in a folder to check for clouds in a specific area.

    Args:
        folder_path (str): The path to the folder with .grb files.
        center_row (int): The row index for the center of the analysis box.
        center_col (int): The column index for the center of the analysis box.
        box_size (int): The width and height of the box to check.

    Returns:
        pd.DataFrame: A DataFrame containing the filename and cloud status.
    """
    files = [f for f in os.listdir(folder_path) if f.endswith(('.grb', '.bin'))]
    analysis_results = []
    
    # Calculate the boundaries of the box to slice
    offset = box_size // 2
    row_start, row_end = center_row - offset, center_row + offset + 1
    col_start, col_end = center_col - offset, center_col + offset + 1

    print(f"Analyzing {len(files)} files with a {box_size}x{box_size} box...")
    for filename in tqdm(files, desc="Processing files"):
        file_path = os.path.join(folder_path, filename)
        grbs = None # Ensure grbs is defined
        
        try:
            grbs = pygrib.open(file_path)
            cloud_mask = grbs.select(name='Cloud mask')[0]
            
            # Slice the data to get our region of interest
            data_slice = cloud_mask.values[row_start:row_end, col_start:col_end]
            
            # In the cloud mask, a value of '2' indicates a cloud.
            # We check if any pixel in our box is a cloud.
            is_cloudy = np.any(data_slice == 2)
            
            analysis_results.append({
                'filename': filename,
                'status': "Cloudy" if is_cloudy else "Clear"
            })

        except Exception as e:
            print(f"Could not process file {filename}: {e}")
        finally:
            if grbs:
                grbs.close()

    return pd.DataFrame(analysis_results)


# ---- MAIN EXECUTION ---- #

def main():
    """
    Main script to locate a target and analyze a folder of GRIB files for clouds.
    """
    print("Starting cloud detection process...")
    
    # We need to open one file first to figure out the grid layout
    all_files = os.listdir(GRIB_FOLDER)
    if not all_files:
        print(f"Error: No files found in the specified folder: {GRIB_FOLDER}")
        return

    sample_file_path = os.path.join(GRIB_FOLDER, all_files[0])
    print(f"Using sample file to find coordinates: {all_files[0]}")

    grbs = pygrib.open(sample_file_path)
    cloud_mask_dataset = grbs.select(name='Cloud mask')[0]
    lats, lons = cloud_mask_dataset.latlons()
    grbs.close()

    # Find the pixel that corresponds to our Lat/Lon target
    center_row, center_col = find_closest_pixel(lats, lons, TARGET_LAT, TARGET_LON)
    print(f"Target coordinates found at pixel (row, col): ({center_row}, {center_col})\n")

    # Now, process all the files in the folder using these coordinates
    results_df = analyze_grib_files(GRIB_FOLDER, center_row, center_col, ANALYSIS_BOX_SIZE)

    if not results_df.empty:
        # Save the results to a CSV
        results_df.to_csv(OUTPUT_CSV_PATH, index=False)
        print(f"\nAnalysis complete. Results saved to: {OUTPUT_CSV_PATH}")
        print("\n--- Sample Output ---")
        print(results_df.head())
    else:
        print("Analysis finished, but no data was generated.")


if __name__ == "__main__":
    main()